In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import html2text
from collections import Counter

In [2]:
import redis

r = redis.Redis(
    host='localhost',
    port=6379)

In [3]:
h = html2text.HTML2Text()
h.ignore_links = True

In [5]:
objects = pd.read_csv("../data/objects.csv")

/Users/chenyangzhu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,7,9,10,17,18,21,22,23,25,26,29,30,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
objects_company = objects[objects['entity_type'] == "Company"]
objects_company = objects_company[objects_company['status'] == "operating"]
objects_company = objects_company[np.logical_not(objects_company['last_funding_at'].isna())]

In [15]:
objects_company

,id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,c:1,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,2008-05-19,3,39750000.0,2010-09-05,2013-09-18,5,17,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00
19,c:10015,Company,10015,NaN,Fitbit,fitbit,/company/fitbit,health,operating,2007-10-01,...,2013-08-13,5,68069200.0,NaN,NaN,0,14,NaN,2008-08-25 02:16:54,2013-12-04 09:52:42
20,c:100155,Company,100155,NaN,MTPV,mtpv,/company/mtpv,cleantech,operating,2003-01-01,...,2012-01-26,3,10125293.0,2010-01-01,2010-01-01,1,6,arun,2011-09-07 02:01:00,2013-04-10 04:35:04
24,c:10018,Company,10018,NaN,Demandbase,demandbase,/company/demandbase,analytics,operating,2006-01-01,...,2013-03-28,3,33000000.0,2010-05-18,2013-03-01,3,19,jason,2008-08-25 03:35:42,2013-10-23 11:32:49
26,c:100189,Company,100189,NaN,Locatrix Communications,locatrix communications,/company/locatrix-communications,mobile,operating,2003-11-01,...,2003-11-01,1,250000.0,2007-11-01,2007-11-01,1,2,mwhite41,2011-09-07 02:27:33,2013-10-11 09:51:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196532,c:998,Company,998,NaN,Wamba,wamba,/company/wamba,web,operating,2013-05-14,...,2007-09-01,1,4111500.0,2008-01-01,2013-09-09,3,2,initial-importer,2007-09-28 09:37:07,2013-05-29 14:40:12
196537,c:99853,Company,99853,NaN,ChatterBlock,chatterblock,/company/chatterblock,social,operating,2011-01-11,...,2011-09-06,1,500000.0,2011-01-11,2011-01-11,1,2,NaN,2011-09-06 22:04:20,2011-09-24 13:07:09
196547,c:9994,Company,9994,NaN,Imindi,imindi,/company/imindi,advertising,operating,2008-09-01,...,2008-05-01,2,500000.0,2009-08-27,2009-08-27,1,2,NaN,2008-08-24 15:40:47,2013-06-28 20:55:06
196549,c:9995,Company,9995,NaN,FohBoh,fohboh,/company/fohboh,public_relations,operating,2007-11-01,...,2008-08-25,1,750000.0,2013-05-01,2013-05-01,1,14,NaN,2008-08-24 15:46:26,2013-06-28 20:55:30


In [16]:
l_website = {}
milestone = Counter()

In [17]:
def store_website(i):
  name = objects_company.iloc[i]['name']
  _id = objects_company.iloc[i]['id']
  founded_at = objects_company.iloc[i]['founded_at']
  
  server = r.get(_id)
  if server is not None:
    if server == "":
      return 0
    else:
      return 1
  
  website_content = ""
  
  # Find website archiv address from Wayback machine
  try:
    # Find company domain from clearbit
    rg = requests.get(f"https://autocomplete.clearbit.com/v1/companies/suggest?query={name}")
    rg_json = rg.json()
    
    if len(rg_json) <= 0:
      milestone['clearbit'] += 1
    else:
      domain = rg.json()[0]['domain'] # Find the most similar domain
      if pd.isna(founded_at):
        wayback_response = requests.get(f"https://archive.org/wayback/available?url={domain}").json()
      else:
        founded_at = founded_at.replace("-", '')
        wayback_response = requests.get(f"https://archive.org/wayback/available?url={domain}&timestamp={founded_at}").json()
      comp_website = wayback_response['archived_snapshots']
      if "closest" in comp_website.keys():
        comp_website = comp_website['closest']
        # Find website:
        if comp_website['status'] == '200':
            website = requests.get(comp_website['url']).content
            website_content = h.handle(website.decode("utf-8"))
  except:
    website_content = ""
      
  r.set(_id, website_content)
  
  if website_content == "":
    return 0
  else:
    return 1

In [18]:
from multiprocessing import Pool
n = objects_company.shape[0]
with Pool(32) as p:
    found = p.map(store_website, [i for i in range(int(n))])

In [26]:
pd.DataFrame(found).to_csv("../data/found_website.csv")

TypeError: to_csv() got an unexpected keyword argument 'axis'

In [32]:
pd.concat([objects_company,
           pd.DataFrame(found, columns=['has_website']).set_index(objects_company.index)], axis=1).to_csv("../data/objects_c.csv")